In [46]:
import numpy as np
import pandas as pd
from fuzzywuzzy import process, fuzz
import time
import math
import re
from langdetect import detect

In [47]:
crs_file=pd.read_excel('../DAC-CRS-CODES-xlsx.xlsx', sheet_name='Purpose codes', header=3)
crs_file_fr=pd.read_excel('../DAC-CRS-CODES-xlsx.xlsx', sheet_name='Codes objet', header=3)

In [49]:
crs_file.columns = ['dac5','crs','voluntary', 'keywords','description','clarification']
crs_file_fr.columns = ['dac5','crs','voluntary','description_fr','clarification_fr']

In [51]:
crs_full_file = pd.merge(crs_file,crs_file_fr,how='left')

In [52]:
crs_full_file.loc[crs_full_file['dac5'].isnull() & crs_full_file['crs'].isnull(),'dac5'] = crs_full_file['voluntary']/100
crs_full_file.loc[crs_full_file['dac5'].isnull()==False & crs_full_file['crs'].isnull(),'crs'] = crs_full_file['voluntary']/10
crs_full_file.loc[crs_full_file['dac5'].isnull() & crs_full_file['voluntary'].isnull(),'dac5'] = crs_full_file['crs']/100

crs_full_file['dac5'] = crs_full_file['dac5'].apply(np.floor)
crs_full_file['crs'] = crs_full_file['crs'].apply(np.floor)

#get rid of 3 digit only rows
crs_full_file = crs_full_file[crs_full_file['crs'].isnull()==False]

In [53]:
crs_full_file['clarification'].replace("'",' ',inplace=True)
crs_full_file.loc[crs_full_file['clarification'].isnull(),'clarification'] = crs_full_file['description']
crs_full_file['description'] = crs_full_file['description'].str.lower()
crs_full_file['clarification'] = crs_full_file['clarification'].str.lower()

crs_full_file['clarification_fr'].replace("'",' ',inplace=True)
crs_full_file.loc[crs_full_file['clarification_fr'].isnull(),'clarification_fr'] = crs_full_file['description_fr']
crs_full_file['description_fr'] = crs_full_file['description_fr'].str.lower()
crs_full_file['clarification_fr'] = crs_full_file['clarification_fr'].str.lower()

crs_desc_l = crs_full_file['description'].str.lower().tolist()
crs_clari_l = crs_full_file['clarification'].str.lower().tolist()

crs_desc_l_fr = crs_full_file['description_fr'].str.lower().tolist()
crs_clari_l_fr = crs_full_file['clarification_fr'].str.lower().tolist()

In [54]:
#
crs_file.loc[crs_file['dac5'].isnull() & crs_file['crs'].isnull(),'dac5'] = crs_file['voluntary']/100
crs_file.loc[crs_file['dac5'].isnull()==False & crs_file['crs'].isnull(),'crs'] = crs_file['voluntary']/10
crs_file.loc[crs_file['dac5'].isnull() & crs_file['voluntary'].isnull(),'dac5'] = crs_file['crs']/100

crs_file_fr.loc[crs_file_fr['dac5'].isnull() & crs_file_fr['crs'].isnull(),'dac5'] = crs_file_fr['voluntary']/100
crs_file_fr.loc[crs_file_fr['dac5'].isnull()==False & crs_file_fr['crs'].isnull(),'crs'] = crs_file_fr['voluntary']/10
crs_file_fr.loc[crs_file_fr['dac5'].isnull() & crs_file_fr['voluntary'].isnull(),'dac5'] = crs_file_fr['crs']/100

In [55]:
#
crs_file['dac5'] = crs_file['dac5'].apply(np.floor)
crs_file['crs'] = crs_file['crs'].apply(np.floor)

crs_file_fr['dac5'] = crs_file_fr['dac5'].apply(np.floor)
crs_file_fr['crs'] = crs_file_fr['crs'].apply(np.floor)

In [67]:
#file to be analysed
####old: 
output_file = pd.read_excel('OECD_sector_review.xlsx', sheet_name='Sheet1')
output_file = output_file.head(200)
output_file.rename(columns = {'8.Description':'project_descr'}, inplace = True)
output_file.rename(columns = {'7.Project title':'project_title'}, inplace = True)
output_file.project_descr = output_file.project_descr.replace( np.nan, ' ').str.lower()
output_file.project_title = output_file.project_title.replace( np.nan, 'x ').str.lower()
output_file['project_descr_long'] = output_file['project_descr'].str.lower()

output_file_deduped = output_file



In [71]:
#new
output_file = pd.read_excel('output_list_2022_02_19.xlsx', sheet_name='Sheet1')
output_file['project_descr'] = output_file['project_descr'].str.lower()
output_file['project_title'] = output_file['project_title'].str.lower()
output_file['project_descr_long'] = output_file['project_descr_long'].str.lower()

In [72]:
output_file.head()

,hq_co,bureau,rollup_ou,project_num,output,project_title,project_descr,project_descr_long,crs,crs_descr
0,CO,RBA,AGO,42722,49533,organizational effectiveness,the purpose of this project is to pay staff sa...,the purpose of this project is to pay staff sa...,91010.0,Administrative costs (non-sector allocable)
1,CO,RBA,AGO,78044,88535,expansion strengthening of a,second gef intervention within a more comprehe...,1. operationalising the protected areas expans...,41010.0,Environmental policy and administrative manage...
2,CO,RBA,AGO,81003,90473,ldcf/adaptation cuvelai rb,promoting climate-resilient development and en...,the development of the province of cunene’s ca...,41010.0,Environmental policy and administrative manage...
3,CO,RBA,AGO,84488,92469,promotion of sustainable charcoal,the project aims to introduce energy-efficient...,1. information and strengthening of the policy...,41010.0,Environmental policy and administrative manage...
4,CO,RBA,AGO,90310,96137,strengthening national capacities,the main goal of this project is to provide si...,•\tstrengthen the national capacity in informa...,16050.0,Multisector aid for basic social services


In [73]:

output_file.project_descr = output_file.project_descr.replace( np.nan, ' ')
output_file.project_title = output_file.project_title.replace( np.nan, 'x ')

output_file_deduped = output_file

In [74]:
#output_file_deduped = output_file.loc[output_file['output'].duplicated()==False]
output_file.head(3)

,hq_co,bureau,rollup_ou,project_num,output,project_title,project_descr,project_descr_long,crs,crs_descr
0,CO,RBA,AGO,42722,49533,organizational effectiveness,the purpose of this project is to pay staff sa...,the purpose of this project is to pay staff sa...,91010.0,Administrative costs (non-sector allocable)
1,CO,RBA,AGO,78044,88535,expansion strengthening of a,second gef intervention within a more comprehe...,1. operationalising the protected areas expans...,41010.0,Environmental policy and administrative manage...
2,CO,RBA,AGO,81003,90473,ldcf/adaptation cuvelai rb,promoting climate-resilient development and en...,the development of the province of cunene’s ca...,41010.0,Environmental policy and administrative manage...


In [75]:
#
crs_file['clarification'].replace("'",' ',inplace=True)
crs_file.loc[crs_file['clarification'].isnull(),'clarification'] = crs_file['description']
crs_file['description'] = crs_file['description'].str.lower()
crs_file['clarification'] = crs_file['clarification'].str.lower()
crs_desc_l = crs_file['description'].str.lower().tolist()
crs_clari_l = crs_file['clarification'].str.lower().tolist()

crs_file_fr['clarification_fr'].replace("'",' ',inplace=True)
crs_file_fr.loc[crs_file_fr['clarification_fr'].isnull(),'clarification_fr'] = crs_file_fr['description_fr']
crs_file_fr['description_fr'] = crs_file_fr['description_fr'].str.lower()
crs_file_fr['clarification_fr'] = crs_file_fr['clarification_fr'].str.lower()
crs_desc_l_fr = crs_file_fr['description_fr'].str.lower().tolist()
crs_clari_l_fr = crs_file_fr['clarification_fr'].str.lower().tolist()

In [76]:
output_file_deduped['project_descr'][7].replace('development','').replace('designed','').replace('support','').replace('conference','').replace('project','')

'this output aims to  the preparation of angolan women to actively participate and be effectively represented in the local governance elections of 2020 as candidates, mobilizers and influencers.'

In [77]:
process.extract(output_file_deduped['project_descr'][7].replace('development','').replace('designed','').replace('support','').replace('conference','').replace('project',''), crs_desc_l, scorer=fuzz.partial_ratio)

[('elections', 100),
 ('coal', 75),
 ('site preservation', 71),
 ('education', 67),
 ('local government finance', 67)]

In [96]:
crs_file.loc[any(ext in output_file_deduped['project_descr'][i] for ext in crs_file['keywords'].split(', ') )]


AttributeError: 'Series' object has no attribute 'split'

In [110]:
crs_file['keywords'] = ' ' + crs_file['keywords']

In [111]:
crs_file.loc[crs_file.apply(lambda x:any(ext in output_file_deduped['project_descr'][i] for ext in str(x['keywords']).split(',') ) , axis=1)]

,dac5,crs,voluntary,keywords,description,clarification
103,151.0,15160.0,NaN,human rights,human rights,measures to support specialised official human...
104,151.0,15170.0,NaN,women,"women’s rights organisations and movements, an...","support for feminist, women-led and women’s ri..."
105,151.0,15180.0,NaN,"women, girls",ending violence against women and girls,support to programmes designed to prevent and ...
201,250.0,25010.0,NaN,"business,",business policy and administration,public sector policies and institution support...


In [118]:
keyword_crs

,dac5,crs,voluntary,keywords,description,clarification
201,250.0,25010.0,NaN,"business,",business policy and administration,public sector policies and institution support...


In [115]:
start_time = time.perf_counter()
#english
for i in output_file_deduped.index.tolist()[0:100]:
#for i in output_file_deduped.index.tolist():
    
    #description checker agains desc
    try:
        if detect(output_file_deduped['project_descr'][i]) =='fr':
            keyword_crs = crs_desc_l_fr
            desc = keyword_crs
            desc_col = 'description_fr'
        else:
            #desc = crs_desc_l
            keyword_crs = crs_file.loc[crs_file.apply(lambda x:any(ext in output_file_deduped['project_descr'][i] for ext in str(x['keywords']).split(',') ) , axis=1)]
            if len(keyword_crs)>0:
                desc = keyword_crs['description'].str.lower().tolist()
            else:
                desc = crs_desc_l
            desc_col = 'description'
    except:
            keyword_crs = crs_file.loc[crs_file.apply(lambda x:any(ext in output_file_deduped['project_descr'][i] for ext in str(x['keywords']).split(',') ) , axis=1)]
            if len(keyword_crs)>0:
                desc = keyword_crs['description'].str.lower().tolist()
            else:
                desc = crs_desc_l
            desc_col = 'description'      
    
    result = process.extract(output_file_deduped['project_descr'][i].replace('development','').replace('designed',''), desc, scorer=fuzz.token_sort_ratio)
    df_result = pd.merge(crs_full_file,pd.DataFrame(result,columns=[desc_col,'descr_score']), 
             how='left', on=desc_col)

    #title checker agains desc
    try:
        if detect(output_file_deduped['project_title'][i]) =='fr':
            desc = crs_desc_l_fr
            desc_col = 'description_fr'
        else:
            desc = crs_desc_l
            desc_col = 'description'
    except:
            desc = crs_desc_l
            desc_col = 'description'        
    
    
    result = process.extract(output_file_deduped['project_title'][i].replace('development','').replace('designed',''), desc, scorer=fuzz.partial_ratio)
    df_result = pd.merge(df_result,pd.DataFrame(result,columns=[desc_col,'title_score']), 
             how='left', on=desc_col)
    
    #deduct score if title word not in description
    title_list =  re.sub("[^\w]", " ",  output_file_deduped['project_title'][i].lower()).split()
    df_result['title_score']=df_result.loc[df_result['title_score']>=0].apply(lambda y:  y['title_score'] if any(x in y['description'] for x in title_list) else y['title_score']*.5, axis=1)
    #extra points if title in description
    df_result['title_score']=df_result.loc[df_result['title_score']>=0].apply(lambda y:  y['title_score'] if any(x in y['clarification'] for x in title_list) else y['title_score']*1.1, axis=1)

    
    #evaluate project descr against clarification
    #description checker agains desc
    try:
        if detect(output_file_deduped['project_descr'][i]) =='fr':
            clari = crs_clari_l_fr
            clari_col = 'clarification_fr'
        else:
            clari = crs_clari_l
            clari_col = 'clarification'
    except:
            clari = crs_clari_l
            clari_col = 'clarification'        
        
    result = process.extract(output_file_deduped['project_descr'][i].replace('development','').replace('designed',''), clari, scorer=fuzz.token_sort_ratio)
    df_result = pd.merge(df_result,pd.DataFrame(result,columns=[clari_col,'descr_c_score']), 
             how='left', on=clari_col)
    
    if output_file_deduped['project_descr_long'][i]==output_file_deduped['project_descr_long'][i]:
        #long description checker agains desc
        try:
            if detect(output_file_deduped['project_descr_long'][i]) =='fr':
                clari = crs_clari_l_fr
                clari_col = 'clarification_fr'
            else:
                clari = crs_clari_l
                clari_col = 'clarification'
        except:
            clari = crs_clari_l
            clari_col = 'clarification'
            
        result = process.extract(output_file_deduped['project_descr_long'][i].replace('development','').replace('designed',''), clari, scorer=fuzz.token_set_ratio)
        df_result = pd.merge(df_result,pd.DataFrame(result,columns=[clari_col,'descr_l_score']), 
             how='left', on=clari_col)
    else:
        df_result['descr_l_score'] = math.nan

    df_result['total_score'] = df_result['descr_score'].fillna(0)*1.25 + df_result['title_score'].fillna(0)*.5 + \
                df_result['descr_l_score'].fillna(0)*.75 + df_result['descr_c_score'].fillna(0)

    #full board match get's 90% average weighting
    df_result.loc[(df_result['descr_score'].isnull() == False) & 
                  (df_result['descr_c_score'].isnull() == False) & 
                  (df_result['title_score'].isnull() == False) & 
                  (df_result['descr_l_score'].isnull() == False),'total_score'] = df_result['total_score']*.9

    #simple double occurance get's 70% average weighting
    df_result.loc[ (
                  ((df_result['descr_score'].isnull() == False) & 
                  (df_result['title_score'].isnull() == False)) |
                  ((df_result['title_score'].isnull() == False) &
                  (df_result['descr_score'].isnull() == False)) |
                  ((df_result['descr_score'].isnull() == False) &
                  (df_result['descr_l_score'].isnull() == False)) |
                  ((df_result['descr_score'].isnull() == False) &
                  (df_result['descr_c_score'].isnull() == False)) |
                  ((df_result['title_score'].isnull() == False) &
                  (df_result['descr_c_score'].isnull() == False)) |
                  ((df_result['descr_l_score'].isnull() == False) &
                  (df_result['descr_c_score'].isnull() == False)) ),'total_score'] = df_result['total_score']*.7

    df_result.loc[(df_result['crs'] == 32261),'total_score'] = df_result['total_score']*.5
    df_result.loc[(df_result['description'] == df_result['clarification']),'total_score'] = df_result['total_score']*.8
    
    #df_result.loc[(df_result['crs']>0) & 
     #         (df_result['crs']==output_file_deduped['crs'][i]) & 
      #        (df_result['crs']!=99810),'total_score'] = df_result['total_score']*1.35

    output_file_deduped.at[i,'dac5_result'] = df_result.loc[df_result['total_score'].argmax()]['dac5']
    output_file_deduped.at[i,'crs_result'] = df_result.loc[df_result['total_score'].argmax()]['crs']
    output_file_deduped.at[i,'volunt_result'] = df_result.loc[df_result['total_score'].argmax()]['voluntary']
    
    if math.isnan(df_result.loc[df_result['total_score'].argmax()]['voluntary'])==False:
        crs_final = df_result.loc[df_result['total_score'].argmax()]['voluntary']
    elif math.isnan(df_result.loc[df_result['total_score'].argmax()]['crs'])==False:
        crs_final = df_result.loc[df_result['total_score'].argmax()]['crs']
    elif math.isnan(df_result.loc[df_result['total_score'].argmax()]['dac5'])==False:
        crs_final = df_result.loc[df_result['total_score'].argmax()]['dac5']
    
    output_file_deduped.at[i,'crs_final'] = crs_final
    output_file_deduped.at[i,'crs_final_2ndplace'] = df_result.loc[df_result['total_score'].nlargest(2).index.tolist()[1]]['crs']
    output_file_deduped.at[i,'crs_final_2ndplace_desc'] = df_result.loc[df_result['total_score'].nlargest(2).index.tolist()[1]]['description']
    output_file_deduped.at[i,'crs_final_3rdplace'] = df_result.loc[df_result['total_score'].nlargest(3).index.tolist()[2]]['crs']
    output_file_deduped.at[i,'crs_final_3rdplace_desc'] = df_result.loc[df_result['total_score'].nlargest(3).index.tolist()[2]]['description']
    output_file_deduped.at[i,'crs_final_4thplace'] = df_result.loc[df_result['total_score'].nlargest(4).index.tolist()[3]]['crs']
    output_file_deduped.at[i,'crs_final_4thplace_desc'] = df_result.loc[df_result['total_score'].nlargest(4).index.tolist()[3]]['description']
    output_file_deduped.at[i,'crs_final_5thplace'] = df_result.loc[df_result['total_score'].nlargest(5).index.tolist()[4]]['crs']
    output_file_deduped.at[i,'crs_final_5thplace_desc'] = df_result.loc[df_result['total_score'].nlargest(5).index.tolist()[4]]['description']
    output_file_deduped.at[i,'crs_final_desc'] = df_result.loc[df_result['total_score'].argmax()]['description']
    output_file_deduped.at[i,'match_score'] = df_result.loc[df_result['total_score'].argmax()]['total_score']
    output_file_deduped.at[i,'title_score'] = df_result.loc[df_result['total_score'].argmax()]['title_score']
    output_file_deduped.at[i,'desc_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_score']
    output_file_deduped.at[i,'desc_l_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_l_score']
    output_file_deduped.at[i,'desc_c_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_c_score']
    
print(time.perf_counter() - start_time, " seconds")

ValueError: 2 columns passed, passed data had 3 columns

In [ ]:
output_file_deduped.head(15)

In [ ]:
output_file_deduped.loc[output_file_deduped['match_score']>=0]

In [20]:
output_file_deduped.to_csv('oecd_review_analysed_2022_02_19.csv')

In [20]:
output_file_deduped = pd.read_csv('oecd_review_analysed_2022_02_19.csv')

In [44]:
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('charcoal')==True,'crs']=23110

output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('election')==True,'crs']=15150
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('human rights')==True,'crs']=15160
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('healthcare')==True,'crs']=13040
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('HIV')==True,'crs']=13040
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('AIDS')==True,'crs']=13040
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('uberculos')==True,'crs']=12263
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('alaria')==True,'crs']=12262
output_file_deduped.loc[output_file_deduped['project_descr'].str.contains('COVID')==True,'crs']=12264
output_file_deduped.loc[(output_file_deduped['project_descr'].str.contains('health')==True)
                        & (output_file_deduped['project_descr'].str.contains('training')==True),'crs']=12261

In [45]:
output_file_deduped.head(20)

,Unnamed: 0,hq_co,bureau,rollup_ou,project_num,output,project_title,project_descr,project_descr_long,crs,...,crs_final_4thplace_desc,crs_final_5thplace,crs_final_5thplace_desc,crs_final_desc,match_score,title_score,desc_score,desc_l_score,desc_c_score,COVID
0,0,CO,RBA,AGO,42722,49533,Organizational Effectiveness,The purpose of this project is to pay staff sa...,The purpose of this project is to pay staff sa...,91010.0,...,other prevention and treatment of ncds,23231.0,solar energy for isolated grids and standalone...,fishery development,57.750,NaN,NaN,50.0,45.0,NaN
1,1,CO,RBA,AGO,78044,88535,Expansion Strengthening of A,Second GEF intervention within a more comprehe...,1. Operationalising the Protected Areas expans...,41010.0,...,ombudsman,1514.0,national standards development,rural land policy and management,43.000,NaN,NaN,NaN,43.0,NaN
2,2,CO,RBA,AGO,81003,90473,LDCF/Adaptation Cuvelai RB,Promoting climate-resilient development and en...,The development of the Province of Cunene’s ca...,41010.0,...,livestock/veterinary services,32210.0,mineral/mining policy and administrative manag...,information services,61.500,NaN,NaN,82.0,NaN,NaN
3,3,CO,RBA,AGO,84488,92469,Promotion of Sustainable Charcoal,The Project aims to introduce energy-efficient...,1. Information and strengthening of the policy...,23110.0,...,national monitoring and evaluation,2201.0,information services,energy conservation and demand-side efficiency,65.100,NaN,40.0,NaN,43.0,NaN
4,4,CO,RBA,AGO,90310,96137,Strengthening National Capacities,The main goal of this project is to provide si...,•\tStrengthen the national capacity in informa...,13040.0,...,judicial affairs,41082.0,environmental research,ombudsman,45.000,NaN,NaN,NaN,45.0,NaN
5,5,CO,RBA,AGO,103650,113597,Support for Local Elections,Technical Assistance for Local Governance (TA-...,The project aims to support the process of ins...,15151.0,...,anti-corruption organisations and institutions,1513.0,judicial affairs,elections,67.900,110.0,NaN,NaN,42.0,NaN
6,6,CO,RBA,AGO,103650,113943,Post-Elections Tech Assistance,Post-Elections Technical Assistance to Angola ...,The project aims to contribute to the strength...,15151.0,...,sectors not specified,12310.0,"ncds control, general",elections,70.700,110.0,NaN,NaN,46.0,NaN
7,7,CO,RBA,AGO,103650,119528,Women for Local Governance,This output aims to support the preparation of...,This output aims to contribute to the empowerm...,15150.0,...,budget planning,15151.0,elections,"women’s rights organisations and movements, an...",50.925,58.0,35.0,NaN,NaN,NaN
8,8,CO,RBA,AGO,106462,107202,Procurement Support Services,Procurement of HIV health products using Gover...,The project aims to strengthen the national HI...,13040.0,...,police,2201.0,postal services,forestry services,70.400,80.0,NaN,64.0,NaN,NaN
9,9,CO,RBA,AGO,106463,107203,Implement Support for IFIGov,Innovative development solutions and partnersh...,1)\tAssess the key capacity needs of various s...,15144.0,...,"women’s rights organisations and movements, an...",1518.0,local government administration,information services,58.500,NaN,NaN,78.0,NaN,NaN


In [39]:
output_file_deduped[output_file_deduped['project_descr'].str.contains('uberculos')==True]

,Unnamed: 0,hq_co,bureau,rollup_ou,project_num,output,project_title,project_descr,project_descr_long,crs,...,crs_final_4thplace_desc,crs_final_5thplace,crs_final_5thplace_desc,crs_final_desc,match_score,title_score,desc_score,desc_l_score,desc_c_score,COVID
30,30,CO,RBA,AGO,136333,128834,C19 RM 2021,The Project seeks to mitigate the effects of t...,The COVID-19 Response Mechanism supports count...,12262.0,...,"facilitation of orderly, safe, regular and res...",13040.0,std control including hiv/aids,low-cost housing,66.000,NaN,NaN,88.0,NaN,12264.0
56,56,CO,RBA,BDI,129553,123180,FM VIH/TB,Le projet vise à soutenir les programmes natio...,La subvention tuberculose et VIH (2021-2023) ...,12263.0,...,std control including hiv/aids,31210.0,forestry policy and administrative management,"social protection and welfare services policy,...",61.425,NaN,31.0,NaN,49.0,NaN
423,423,CO,RBA,COG,129730,123299,"HIV, TB and Capacity Building",La subvention est financée par le Fonds Mondia...,Le projet a pour objectif de lutter contre les...,12263.0,...,forestry policy and administrative management,12182.0,medical research,tuberculosis control,65.250,NaN,NaN,87.0,NaN,NaN
793,793,CO,RBA,LBR,125917,123627,PIP/UNDP/MOH/Global Fund Colla,The PIP/UNDP/MOH/Global Fund Collaboration fra...,The capacity strengthening plan aims to achiev...,12262.0,...,std control including hiv/aids,13081.0,personnel development for population and repro...,livestock/veterinary services,43.500,NaN,NaN,58.0,NaN,12264.0
1776,1776,CO,RBAP,AFG,128299,122298,"HIV/AIDS, TB Malaria","HIV/AIDS, Tuberculosis Malaria Programs and H...",NaN,12262.0,...,sectors not specified,99810.0,sectors not specified,administration of developing countries' foreig...,59.500,NaN,32.0,NaN,45.0,12264.0
1777,1777,CO,RBAP,AFG,128299,129589,CRM 19,"CRM 19, HIV/AIDS, Tuberculosis Malaria Progra...",NaN,12262.0,...,communications policy and administrative manag...,33181.0,trade education/training,administration of developing countries' foreig...,57.925,NaN,31.0,NaN,44.0,12264.0
2047,2047,CO,RBAP,IDN,106768,127136,TAA FOR IMPLEMENTATION GFATM,"UNDP Technical Cooperation for the HIV, Tuberc...",To provide technical assistance to Indonesia’s...,12262.0,...,std control including hiv/aids,41082.0,environmental research,general personnel services,54.750,NaN,NaN,73.0,NaN,12264.0
2300,2300,CO,RBAP,MMR,63637,80622,GLOBAL FUND SECRETARIAT CCM,GLOBAL FUND SECRETARIAT CCM FUNDING to support...,GLOBAL FUND SECRETARIAT CCM FUNDING to support...,12263.0,...,std control including hiv/aids,22010.0,communications policy and administrative manag...,administration of developing countries' foreig...,54.600,NaN,NaN,48.0,42.0,NaN
3173,3173,CO,RBAS,SDN,108620,108353,Continuity of fighting TB,To decrease Tuberculosis (TB) burden in Sudan ...,The main goal of the project is to decrease TB...,12263.0,...,trade education/training,72050.0,relief co-ordination and support services,national monitoring and evaluation,53.250,NaN,NaN,71.0,NaN,NaN
4271,4271,CO,RBEC,UKR,70648,84507,Strengthening Capacity,The capacity of the National Council on Tuberc...,Component 1 - Strengthening of oversight funct...,12262.0,...,tuberculosis control,13030.0,family planning,std control including hiv/aids,53.200,NaN,NaN,44.0,43.0,12264.0


In [43]:
output_file_deduped[['output','fiscal_year','project_title','crs_result','crs_final','crs_final_2ndplace','crs','title_score','desc_score','desc_c_score','desc_l_score','match_score','crs_final_desc']].loc[i]

output                                        81278
fiscal_year                                    2021
project_title             Development Effectiveness
crs_result                                    25030
crs_final                                     25030
crs_final_2ndplace                            99820
crs                                           91010
title_score                                      71
desc_score                                      NaN
desc_c_score                                    NaN
desc_l_score                                    NaN
match_score                                      71
crs_final_desc        business development services
Name: 99, dtype: object

In [44]:
output_file_deduped[['fiscal_year','project_title','crs_final_desc','crs_result','crs_final','crs','title_score','desc_score','desc_c_score','desc_l_score','match_score']].loc[ (output_file_deduped['match_score']>0 )]

,fiscal_year,project_title,crs_final_desc,crs_result,crs_final,crs,title_score,desc_score,desc_c_score,desc_l_score,match_score
0,2021,IONA CONSERVATION/ GEF,energy conservation and demand-side efficiency,23183.0,23183.0,41010.0,80.3,NaN,NaN,NaN,80.300
1,2021,Support Enhancemen Human Right,human rights,15160.0,15160.0,15131.0,96.0,NaN,NaN,NaN,96.000
2,2021,Sust Econ Dev Inclusive Gowth,postal services,2201.0,22012.0,16050.0,NaN,NaN,NaN,58.0,49.300
3,2021,Expansion Strengthening of A,other prevention and treatment of ncds,12350.0,12350.0,41010.0,60.0,NaN,NaN,NaN,60.000
4,2021,LDCF/Adaptation Cuvelai RB,urban land policy and management,4303.0,43031.0,41010.0,NaN,NaN,NaN,91.0,77.350
...,...,...,...,...,...,...,...,...,...,...,...
95,2021,COV02 BEN-00125781 MPTF R2R,"social protection and welfare services policy,...",1601.0,16011.0,16050.0,NaN,46.0,46.0,NaN,64.400
96,2021,BEN COVID-19 Demat Admin Proce,district heating and cooling,23620.0,23620.0,16062.0,NaN,NaN,48.0,NaN,48.000
97,2021,Appui Institutionnel ADSC TEF,personnel development for population and repro...,13081.0,13081.0,16050.0,NaN,38.0,45.0,NaN,58.100
98,2021,Promotion Droits Handicapés 2,trade education/training,33181.0,33181.0,15131.0,NaN,NaN,48.0,45.0,60.375


In [2]:
output_file['project_descr_long'][i] ==output_file['project_descr_long'][i]

NameError: name 'output_file' is not defined

In [ ]:
df_result = pd.merge(crs_file,pd.DataFrame(result,columns=['description','descr_score']), 
             how='left', on='description')

In [ ]:
df_result = pd.merge(df_result,pd.DataFrame(result,columns=['description','title_score']), 
             how='left', on='description')

In [ ]:
df_result = pd.merge(df_result,pd.DataFrame(result,columns=['description','descr_l_score']), 
             how='left', on='description')

In [ ]:
df_result[df_result['total_score']>0]

In [ ]:
df_result[(df_result['descr_score'].isnull() == False) | 
              (df_result['title_score'].isnull() == False) | 
              (df_result['descr_c_score'].isnull() == False) | 
              (df_result['descr_l_score'].isnull() == False)]

In [ ]:
pd.DataFrame(result,columns=[desc_col,'descr_score'])

In [ ]:
if detect(output_file_deduped['project_descr'][i]) =='fr':
    desc = crs_desc_l_fr
    desc_col = 'description_fr'
else:
    desc = crs_desc_l
    desc_col = 'description'

result = process.extract(output_file_deduped['project_descr'][i], desc, scorer=fuzz.token_sort_ratio)
df_result = pd.merge(crs_full_file,pd.DataFrame(result,columns=[desc_col,'descr_score']), 
         how='left', on=desc_col)


In [ ]:

#title checker agains desc
if detect(output_file_deduped['project_title'][i]) =='fr':
    desc = crs_desc_l_fr
    desc_col = 'description_fr'
else:
    desc = crs_desc_l
    desc_col = 'description'

result = process.extract(output_file_deduped['project_title'][i], desc, scorer=fuzz.partial_ratio)
df_result = pd.merge(df_result,pd.DataFrame(result,columns=[desc_col,'title_score']), 
         how='left', on=desc_col)

#deduct score if title word not in description
title_list =  re.sub("[^\w]", " ",  output_file_deduped['project_title'][i].lower()).split()
df_result['title_score']=df_result.loc[df_result['title_score']>=0].apply(lambda y:  y['title_score'] if any(x in y['description'] for x in title_list) else y['title_score']*.5, axis=1)

#evaluate project descr against clarification
#description checker agains desc
if detect(output_file_deduped['project_descr'][i]) =='fr':
    clari = crs_clari_l_fr
    clari_col = 'clarification_fr'
else:
    clari = crs_clari_l
    clari_col = 'clarification'

result = process.extract(output_file_deduped['project_descr'][i], clari, scorer=fuzz.token_set_ratio)
df_result = pd.merge(df_result,pd.DataFrame(result,columns=[clari_col,'descr_c_score']), 
         how='left', on=clari_col)

if output_file_deduped['project_descr_long'][i]==output_file_deduped['project_descr_long'][i]:
    #long description checker agains desc
    if detect(output_file_deduped['project_descr_long'][i]) =='fr':
        clari = crs_clari_l_fr
        clari_col = 'clarification_fr'
    else:
        clari = crs_clari_l
        clari_col = 'clarification'

    result = process.extract(output_file_deduped['project_descr_long'][i], clari, scorer=fuzz.token_set_ratio)
    df_result = pd.merge(df_result,pd.DataFrame(result,columns=[clari_col,'descr_l_score']), 
         how='left', on=clari_col)
else:
    df_result['descr_l_score'] = math.nan

df_result['total_score'] = df_result['descr_score'].fillna(0) + df_result['title_score'].fillna(0) + \
            df_result['descr_l_score'].fillna(0) + df_result['descr_c_score'].fillna(0)

df_result.loc[(df_result['descr_score'].isnull() == False) & 
              (df_result['descr_c_score'].isnull() == False) & 
              (df_result['title_score'].isnull() == False) & 
              (df_result['descr_l_score'].isnull() == False),'total_score'] = df_result['total_score']*1.1


df_result.loc[((df_result['descr_score'].isnull() == False) & 
              (df_result['title_score'].isnull() == False)) |
              ((df_result['title_score'].isnull() == False) &
              (df_result['descr_score'].isnull() == False)) |
              ((df_result['descr_score'].isnull() == False) &
              (df_result['descr_l_score'].isnull() == False)) |
              ((df_result['descr_score'].isnull() == False) &
              (df_result['descr_c_score'].isnull() == False)) |
              ((df_result['title_score'].isnull() == False) &
              (df_result['descr_c_score'].isnull() == False)) |
              ((df_result['descr_l_score'].isnull() == False) &
              (df_result['descr_c_score'].isnull() == False)),'total_score'] = df_result['total_score']*1.05

df_result.loc[(df_result['crs']>0) & 
          (df_result['crs']==output_file_deduped['crs'][i]) & 
          (df_result['crs']!=99810),'total_score'] = df_result['total_score']*1.35

output_file_deduped.at[i,'dac5_result'] = df_result.loc[df_result['total_score'].argmax()]['dac5']
output_file_deduped.at[i,'crs_result'] = df_result.loc[df_result['total_score'].argmax()]['crs']
output_file_deduped.at[i,'volunt_result'] = df_result.loc[df_result['total_score'].argmax()]['voluntary']

if math.isnan(df_result.loc[df_result['total_score'].argmax()]['voluntary'])==False:
    crs_final = df_result.loc[df_result['total_score'].argmax()]['voluntary']
elif math.isnan(df_result.loc[df_result['total_score'].argmax()]['crs'])==False:
    crs_final = df_result.loc[df_result['total_score'].argmax()]['crs']
elif math.isnan(df_result.loc[df_result['total_score'].argmax()]['dac5'])==False:
    crs_final = df_result.loc[df_result['total_score'].argmax()]['dac5']

output_file_deduped.at[i,'crs_final'] = crs_final
output_file_deduped.at[i,'crs_final_2ndplace'] = df_result.loc[df_result['total_score'].nlargest(2).index.tolist()[1]]['crs']
output_file_deduped.at[i,'crs_final_desc'] = df_result.loc[df_result['total_score'].argmax()]['description']
output_file_deduped.at[i,'match_score'] = df_result.loc[df_result['total_score'].argmax()]['total_score']
output_file_deduped.at[i,'title_score'] = df_result.loc[df_result['total_score'].argmax()]['title_score']
output_file_deduped.at[i,'desc_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_score']
output_file_deduped.at[i,'desc_l_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_l_score']
output_file_deduped.at[i,'desc_c_score'] = df_result.loc[df_result['total_score'].argmax()]['descr_c_score']


In [28]:
df_result

,dac5,crs,voluntary,description,clarification,description_fr,clarification_fr,descr_score,title_score,descr_c_score,descr_l_score,total_score
0,111.0,11110.0,NaN,education policy and administrative management,"education sector policy, planning and programm...",politique de l’éducation et gestion administra...,"politique de l’éducation, planification et pro...",NaN,NaN,NaN,NaN,0.0
1,111.0,11120.0,NaN,education facilities and training,"educational buildings, equipment, materials; s...",equipements scolaires et formation,"bâtiments scolaires, équipement, fournitures ;...",NaN,NaN,NaN,NaN,0.0
2,111.0,11130.0,NaN,teacher training,teacher education (where the level of educatio...,formation des enseignants,éducation des enseignants (quand le niveau d’é...,NaN,NaN,NaN,NaN,0.0
3,111.0,11182.0,NaN,educational research,research and studies on education effectivenes...,recherche en éducation,"recherche et études sur l’efficacité, la perti...",NaN,NaN,NaN,NaN,0.0
4,112.0,11220.0,NaN,primary education,formal and non-formal primary education for ch...,enseignement primaire,enseignement primaire formel et non formel pou...,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
290,930.0,9301.0,93016.0,refugees/asylum seekers in donor countries - t...,costs incurred in donor countries for basic as...,refugiés/demandeurs d’asile dans les pays donn...,coûts encourus dans les pays donneurs au titre...,NaN,NaN,NaN,NaN,0.0
291,930.0,9301.0,93017.0,refugees/asylum seekers in donor countries - r...,costs incurred in donor countries for basic as...,refugiés/demandeurs d’asile dans les pays donn...,coûts encourus dans les pays donneurs au titre...,NaN,NaN,NaN,NaN,0.0
292,930.0,9301.0,93018.0,refugees/asylum seekers in donor countries - a...,costs incurred in donor countries for basic as...,refugiés/demandeurs d’asile dans les pays donn...,coûts encourus dans les pays donneurs au titre...,NaN,NaN,NaN,NaN,0.0
293,998.0,99810.0,NaN,sectors not specified,contributions to general development of the re...,secteur non spécifié,les contributions au développement général du ...,NaN,NaN,NaN,NaN,0.0
